In [3]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,3):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.coolmod.com/portatiles-portatiles/?pagina=1{i}")
    
    articles = r.html.find('article')
    links = [article.find('a', first=True).attrs['href'] for article in articles]
    portatil_links.extend(links)

Pagina 1
Pagina 2


In [4]:
portatil_links[0]

'/msi-creator-pro-16-ai-studio-a1vjg-209es-u9-185h-rtx-2000-32gb-2tb-16-w11p/'

In [41]:
portatil_features = []

for link in portatil_links:
    r = session.get(url=f'https://www.coolmod.com{link}')
    specs = r.html.find('.collapse-content.prose', first=True)

    features = {}

    # Cada característica del portátil
    elements = specs.find('p, ul')

    current_section = None
    for element in elements:
        if element.tag == 'p':
            # Inicia una nueva sección
            current_section = element.text.strip()
            features[current_section] = []
        elif element.tag == 'ul' and current_section:
            # Características asociadas a la sección actual
            for li in element.find('li'):
                text = li.text.strip()
                if ':' in text:
                    key, value = text.split(':', 1)
                    features[current_section].append({key.strip(): value.strip()})
                else:
                    features[current_section].append(text)
        elif element.tag == 'ul':
            # Características sin sección
            for li in element.find('li'):
                text = li.text.strip()
                if ':' in text:
                    key, value = text.split(':', 1)
                    features[key.strip()] = value.strip()
    
    portatil_features.append(features)

In [42]:
portatil_features

[{'Procesador': [{'Modelo': 'Intel® Core™ Ultra 9 processor 185H con Intel® AI Boost (NPU)'},
   {'Cores': '16 (6 P-cores + 8 E-cores + 2 Low Power E-cores)'},
   {'Frecuencia Turbo Máxima': '5.1 GHz'}],
  'Sistema Operativo': [{'Versión': 'Windows 11 Pro'}],
  'Versión: Windows 11 Pro': [],
  'Pantalla': [{'Tamaño': '16" UHD+ (3840x2400)'},
   {'Frecuencia de Actualización': '120Hz'},
   {'Tipo': 'Mini LED, IPS-Level'},
   {'Certificación': 'VESA DisplayHDR™ 1000'},
   {'Cobertura de Color': '100% DCI-P3 (Típico)'}],
  'Chipset': [{'Tipo': 'Integrated SoC'}],
  'Tipo: Integrated SoC': [],
  'Gráficos': [{'GPU': 'NVIDIA RTX™ 2000 Ada Laptop GPU (8GB GDDR6)'},
   {'AI TOPS': '232'},
   {'Reloj Boost': 'Hasta 2340MHz'},
   {'Potencia Gráfica Máxima': '105W con Dynamic Boost (puede variar según el escenario)'}],
  'Memoria': [{'Memoria Total': '16GB*2, DDR5-5600'},
   {'Capacidad Máxima': '96GB en 2 Slots (puede variar según la capacidad de memoria)'}],
  'Almacenamiento': [{'Tipo': '2TB 

In [44]:
import re

def extract_memory_size(text):
    # Busca patrones como "16GB", "16 GB", "32GB*2"
    match = re.search(r'(\d+)\s*GB', str(text))
    if match:
        return match.group(1)
    return ''

def extract_storage_size(text):
    # Busca patrones como "512GB", "1TB", "2TB"
    match = re.search(r'(\d+)\s*(TB|GB)', str(text))
    if match:
        size = int(match.group(1))
        unit = match.group(2)
        if unit == 'TB':
            size *= 1024
        return str(size)
    return ''

def extract_screen_size(text):
    # Busca patrones como "16"", "15.6"", "14""
    match = re.search(r'(\d+\.?\d?)[""]', str(text))
    if match:
        return match.group(1)
    return ''

def extract_processor_model(text):
    # Busca el modelo del procesador
    match = re.search(r'Intel.*?(?=;)|AMD.*?(?=;)|Apple.*?(?=;)', str(text))
    if match:
        model = match.group(0)
        # Limpieza del texto
        model = re.sub(r'[®™]', '', model)  # Elimina ® y ™
        model = re.sub(r'\s+with.*|\s+con.*|\(.*?\)', '', model)  # Elimina texto después de "with" o "con" y paréntesis
        model = re.sub(r'\s+processor', '', model)  # Elimina "processor"
        model = re.sub(r'\s+de\s+\d+\.ª\s+generación.*', '', model)  # Elimina generación y texto posterior
        model = re.sub(r',.*', '', model)  # Elimina todo después de la coma
        model = ' '.join(model.split())  # Elimina espacios extra
        return model
    return ''

def extract_graphics_card(text):
    # Busca la tarjeta gráfica
    match = re.search(r'(NVIDIA.*?|AMD.*?|Intel.*?)(?=;|\(|$)', str(text))
    if match:
        return match.group(1)
    return ''

def extract_battery_capacity(text):
    # Busca la capacidad de la batería
    match = re.search(r'(\d+(?:\.\d+)?)\s*Whrs?', str(text))
    if match:
        return match.group(1)
    return ''

def extract_weight(text):
    # Busca el peso en kg
    match = re.search(r'(\d+(?:\.\d+)?)\s*kg', str(text))
    if match:
        return match.group(1)
    return ''

def extract_os_version(text):
    # Busca la versión del sistema operativo
    match = re.search(r'(Windows \d+\s*\w*|macOS|Linux|FreeDOS)', str(text))
    if match:
        return match.group(1)
    return ''

In [46]:
import csv

# Definir las características que queremos almacenar en el CSV
deseadas = {
    'Procesador': 'Procesador',
    'Memoria': 'Memoria RAM',
    'Almacenamiento': 'Almacenamiento',
    'Gráficos': 'Tarjeta Gráfica',
    'Pantalla': 'Pantalla',
    'Batería': 'Batería',
    'Peso': 'Peso',
    'Sistema Operativo': 'Sistema Operativo',
    'Conectividad': 'Conectividad'
}

columnas = list(deseadas.values()) + ['Tamaño de Pantalla']

# Abrir el archivo CSV para escritura
with open('caracteristicas.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=columnas)
    writer.writeheader()

    for features in portatil_features:
        salida = {}
        for clave_origen, clave_csv in deseadas.items():
            if clave_origen in features:
                valor = features[clave_origen]
                if isinstance(valor, list):
                    partes = []
                    for item in valor:
                        if isinstance(item, dict):
                            partes.extend([f"{k}: {v}" for k, v in item.items()])
                        else:
                            partes.append(str(item))
                    texto_completo = "; ".join(partes)
                    
                    # Procesamiento especial para cada característica
                    if clave_origen == 'Memoria':
                        salida[clave_csv] = extract_memory_size(texto_completo)
                    elif clave_origen == 'Almacenamiento':
                        salida[clave_csv] = extract_storage_size(texto_completo)
                    elif clave_origen == 'Procesador':
                        salida[clave_csv] = extract_processor_model(texto_completo)
                    elif clave_origen == 'Gráficos':
                        salida[clave_csv] = extract_graphics_card(texto_completo)
                    elif clave_origen == 'Batería':
                        salida[clave_csv] = extract_battery_capacity(texto_completo)
                    elif clave_origen == 'Peso':
                        salida[clave_csv] = extract_weight(texto_completo)
                    elif clave_origen == 'Sistema Operativo':
                        salida[clave_csv] = extract_os_version(texto_completo)
                    else:
                        salida[clave_csv] = texto_completo
                else:
                    salida[clave_csv] = str(valor)
            else:
                salida[clave_csv] = ''

        # Extraer tamaño de pantalla si está disponible en la información de pantalla
        if 'Pantalla' in features:
            pantalla_texto = "; ".join([str(item) for item in features['Pantalla']])
            salida['Tamaño de Pantalla'] = extract_screen_size(pantalla_texto)
        else:
            salida['Tamaño de Pantalla'] = ''

        # Escribir la fila en el archivo CSV
        writer.writerow(salida)

In [ ]:
import json
from requests_html import HTMLSession

session = HTMLSession()
info_list = []

for link in portatil_links:
    r = session.get(f'https://www.mediamarkt.es{link}')
    tables = r.html.find('td')
    
    info = {}
    for i in range(0, len(tables), 2):
        clave = tables[i].text.strip()
        valor = tables[i+1].text.strip()
        info[clave] = valor
    
    info_list.append(info)

with open('info_list.json', 'w', encoding='utf-8') as f:
    json.dump(info_list, f, ensure_ascii=False, indent=4)

for info in info_list:
    print(info)